# Engenius '20 - EDA

## Summary

The Main goal is to predict the quantities sold of a set of products within a 4 months time frame.

# Table Of Contents

# Exploratory Data Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

product_sheets = ['PRODUCT_A', 'PRODUCT_B', 'PRODUCT_C', 'PRODUCT_V', 'PRODUCT_X']
data = {}

for s in product_sheets:
    data[s] = pd.read_excel('datasets/Data2017-2019.xls', s)
    


In [2]:
# merge data frames for all 5 products vertically
df_all5 = data[product_sheets[0]]
for i,s in enumerate(product_sheets):
    if i != 0:
        df_all5 = df_all5.append(data[s], ignore_index=True)


In [3]:
df_all5.shape

(58032, 5)

In [4]:
df_all5.Product.value_counts()

PRODUCT_C    17980
PRODUCT_B    17962
PRODUCT_A    17371
PRODUCT_X     2920
PRODUCT_V     1799
Name: Product, dtype: int64

### Null Valued Features
- ratio of null values
  No null values as seen below

In [5]:
(df_all5.isnull().sum() / df_all5.shape[0])

Year        0.0
Period      0.0
Product     0.0
Province    0.0
Quantity    0.0
dtype: float64

* ratio of transactions (sales) with negative quantities
  probably product return

In [6]:
(df_all5[df_all5.Quantity < 0].shape[0] / df_all5.shape[0])

0.0016197959746346844

In [7]:
data_sorted = df_all5.sort_values(by=['Period', 'Province', 'Product']).reset_index()
data_sorted.head(5)

,index,Year,Period,Product,Province,Quantity
0,4,2017,201701,PRODUCT_A,ADANA,11
1,5,2017,201701,PRODUCT_A,ADANA,25
2,7,2017,201701,PRODUCT_A,ADANA,54
3,8,2017,201701,PRODUCT_A,ADANA,17
4,9,2017,201701,PRODUCT_A,ADANA,30


In [8]:
data_sorted = df_all5.sort_values(by=['Period', 'Province', 'Product']).reset_index()
data_sorted.head(5)

,index,Year,Period,Product,Province,Quantity
0,4,2017,201701,PRODUCT_A,ADANA,11
1,5,2017,201701,PRODUCT_A,ADANA,25
2,7,2017,201701,PRODUCT_A,ADANA,54
3,8,2017,201701,PRODUCT_A,ADANA,17
4,9,2017,201701,PRODUCT_A,ADANA,30


In [9]:
data_sorted.groupby(['Period', 'Province', 'Product'])['Quantity'].sum()

Period  Province   Product  
201701  ADANA      PRODUCT_A     346
                   PRODUCT_B    4838
                   PRODUCT_C    1004
                   PRODUCT_V      20
                   PRODUCT_X     910
                                ... 
201912  ŞANLIURFA  PRODUCT_X     317
        ŞIRNAK     PRODUCT_A      28
                   PRODUCT_B     144
                   PRODUCT_C     323
                   PRODUCT_X      27
Name: Quantity, Length: 13523, dtype: int64